In [1]:
import stackstac
import numpy as np
import torch
import planetary_computer
from pystac_client import Client

In [2]:
# Access the Planetary Computer's STAC API
stac_api_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
client = Client.open(stac_api_url)

# Search for Sentinel-2 imagery
search = client.search(
    collections=["sentinel-2-l2a"],
    bbox=[-106.5, 35.0, -106.4, 35.1],  # Define a bounding box (e.g., around a location)
    datetime="2023-01-01/2023-12-31",  # Specify time range for NDVI time series
    query={"eo:cloud_cover": {"lt": 20}},  # Filter by cloud cover
)

# Get the items from the search
items = search.item_collection()

# Sign the items for authenticated access
signed_items = [planetary_computer.sign(item).to_dict() for item in items]

# Stackstac to read the imagery into an xarray DataArray
datacube = stackstac.stack(
    signed_items,
    assets=["B04", "B08"],  # Red and NIR bands for NDVI
    resolution=30,
    dtype=np.float64,
    bounds_latlon=(-106.5, 35.0, -106.4, 35.1), 
    epsg=3857,
    rescale=False  
)

# Use labeled dimensions
datacube = datacube.rename(band="band", x="longitude", y="latitude", time="time")
datacube 

<xarray.DataArray 'stackstac-0eadfd8ffe686c8acc21537811efcf1f' (time: 96,
                                                                band: 2,
                                                                latitude: 454,
                                                                longitude: 372)> Size: 259MB
dask.array<fetch_raster_window, shape=(96, 2, 454, 372), dtype=float64, chunksize=(1, 1, 454, 372), chunktype=numpy.ndarray>
Coordinates: (12/46)
  * time                                     (time) datetime64[ns] 768B 2023-...
    id                                       (time) <U54 21kB 'S2B_MSIL2A_202...
  * band                                     (band) <U3 24B 'B04' 'B08'
  * longitude                                (longitude) float64 3kB -1.186e+...
  * latitude                                 (latitude) float64 4kB 4.178e+06...
    s2:product_type                          <U7 28B 'S2MSI2A'
    ...                                       ...
    proj:transform                           object 8B {0.0, 3900000.0, 30000...
    gsd                                      float64 8B 10.0
    common_name                              (band) <U3 24B 'red' 'nir'
    center_wavelength                        (band) float64 16B 0.665 0.842
    full_width_half_max                      (band) float64 16B 0.038 0.145
    epsg                                     int64 8B 3857
Attributes:
    spec:        RasterSpec(epsg=3857, bounds=(-11855550, 4163880, -11844390,...
    crs:         epsg:3857
    transform:   | 30.00, 0.00,-11855550.00|\n| 0.00,-30.00, 4177500.00|\n| 0...
    resolution:  30

In [3]:
#  resample to monthly intervals
datacube_monthly = datacube.resample(time="1ME").median()

In [4]:
# Convert Xarray DataArray to NumPy array
datacube_numpy = datacube_monthly.values


In [5]:
datacube_numpy

array([[[[2071. , 2237. , 2723. , ..., 1471. , 1462. , 1529. ],
         [1833.5, 2346. , 2433. , ..., 1451.5, 1512.5, 1506.5],
         [2027. , 2282. , 2282. , ..., 1577.5, 1522. , 1472.5],
         ...,
         [1402.5, 1376. , 1384. , ..., 1865. , 1898. , 2118. ],
         [2276. , 2673. , 2563. , ..., 1824.5, 1968. , 2259. ],
         [2604. , 2554. , 2473. , ..., 1915. , 2052. , 2360. ]],

        [[2506. , 2891. , 3052. , ..., 2470. , 2532. , 2308. ],
         [2573. , 2903. , 2710. , ..., 2486. , 2493. , 2456. ],
         [2467. , 3073. , 2648. , ..., 2488. , 2418. , 2505. ],
         ...,
         [1516. , 1515.5, 1525.5, ..., 2695. , 2751. , 2939. ],
         [2602. , 3008. , 2974. , ..., 2696. , 2731. , 2947. ],
         [2907. , 2848. , 2798. , ..., 2727. , 2866. , 2926. ]]],


       [[[2040. , 2466. , 2938. , ..., 1430. , 1388. , 1476. ],
         [1948. , 2368. , 2630. , ..., 1353. , 1454. , 1436. ],
         [2112. , 2306. , 2290. , ..., 1536. , 1478. , 1457. ],
      

In [6]:
# Convert NumPy array to PyTorch tensor
datacube_tensor = torch.from_numpy(datacube_numpy)

In [7]:
#  reorder dimensions for PyTorch (if needed)
#datacube_tensor = datacube_tensor.permute(0, 2, 3, 1)

In [8]:
datacube_tensor.shape

torch.Size([12, 2, 454, 372])

In [9]:
datacube_tensor.dtype

torch.float64